In [7]:
from transformers import pipeline

c:\Users\lhars\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
ner_tagger = pipeline("ner", model="dslim/bert-base-NER", aggregation_strategy="simple")

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

Loading weights: 100%|██████████| 199/199 [00:01<00:00, 172.64it/s, Materializing param=classifier.weight]                                      
BertForTokenClassification LOAD REPORT from: dslim/bert-base-NER
Key                      | Status     |  | 
-------------------------+------------+--+-
bert.pooler.dense.weight | UNEXPECTED |  | 
bert.pooler.dense.bias   | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
Loading weights: 100%|██████████| 515/515 [00:01<00:00, 484.72it/s, Materializing param=model.shared.weight]                                   


In [9]:
def parse_resume_snippet(text):
    candidate_labels = ["Education", "Work Experience", "Technical Skills", "Contact Information"]
    
    category_result = classifier(text, candidate_labels=candidate_labels)
    top_category = category_result['labels'][0]
    confidence = category_result['scores'][0]
    
    entities = ner_tagger(text)
    
    structured_entities = [
        {"word": ent['word'], "label": ent['entity_group'], "score": round(ent['score'], 3)} 
        for ent in entities
    ]
    
    return {
        "detected_section": top_category,
        "confidence_score": round(confidence, 3),
        "entities_found": structured_entities
    }

In [12]:
test_resume = """
RELEVANT EXPERIENCE
Lead NLP Engineer | TechFlow Solutions, San Francisco | Jan 2022 - Present
- Orchestrated the deployment of a Retrieval-Augmented Generation (RAG) pipeline using LangChain and Pinecone.
- Scaled transformer-based microservices on AWS SageMaker, reducing latency by 40%.
- Mentored a team of 5 junior developers in Agile sprints.

ACADEMIC BACKGROUND
Ph.D. in Computational Linguistics
Massachusetts Institute of Technology (MIT), Cambridge
GPA: 3.9/4.0 | 2017 - 2021

TECHNICAL EXPERTISE
Languages: Python, Rust, C++, SQL.
Frameworks: PyTorch, Hugging Face Transformers, Docker, Kubernetes.
"""

In [13]:
result = parse_resume_snippet(test_resume)

print(f"--- PARSING RESULTS ---")
print(f"Predicted Section: {result['detected_section']} ({result['confidence_score']})")
print(f"\nEntities Extracted:")
for entity in result['entities_found']:
    print(f"- {entity['word']} ({entity['label']})")

--- PARSING RESULTS ---
Predicted Section: Technical Skills (0.699)

Entities Extracted:
- TechFlow Solutions (ORG)
- San Francisco (LOC)
- Re (MISC)
- Generation (MISC)
- LangChain (ORG)
- Pinecone (ORG)
- A (MISC)
- ##M (ORG)
- ACADEMIC (ORG)
- Computational (MISC)
- Linguistics (ORG)
- Massachusetts Institute of Technology (ORG)
- MIT (ORG)
- Cambridge (LOC)
- Python (MISC)
- R (MISC)
- C (MISC)
- SQL (MISC)
- P (ORG)
- ##Tor (ORG)
- Face (ORG)
- Dock (ORG)
- Ku (ORG)
- ##net (ORG)
